In [9]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from langchain_community.embeddings import HuggingFaceEmbeddings

In [4]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
# Create a new index -> metric: cosine (finds answers based on cosine similarity), dimension: 128 (each vector has 128 dimensions)
# pc.create_index(name="rag", metric="cosine", dimension=128, spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     )) 

In [5]:
import json 
data = json.load(open("reviews.json"))
data["reviews"]

[{'professor': 'Dr. Emily Chen',
  'rating': 4,
  'subject': 'Biology',
  'review': "Dr. Chen's lectures are engaging and well-structured. She's always willing to help during office hours."},
 {'professor': 'Prof. Michael Johnson',
  'rating': 3,
  'subject': 'History',
  'review': 'Prof. Johnson knows his subject well, but his lectures can be dry at times. Assignments are fair.'},
 {'professor': 'Dr. Sarah Williams',
  'rating': 5,
  'subject': 'Computer Science',
  'review': 'Brilliant professor! Dr. Williams explains complex concepts clearly and provides great real-world examples.'},
 {'professor': 'Prof. David Lee',
  'rating': 2,
  'subject': 'Mathematics',
  'review': "Prof. Lee's teaching style is confusing. Often seems unprepared for class and struggles to answer questions."},
 {'professor': 'Dr. Amanda Rodriguez',
  'rating': 4,
  'subject': 'Psychology',
  'review': 'Dr. Rodriguez is passionate about her field and it shows. Her classes are thought-provoking and interactive.'}

In [15]:
processed_data = []
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY")
)

for review in data["reviews"]:
    try:
        embeddingResponse = await client.embeddings.create(
            model='huggingfaceh4/zephyr-7b-beta:free',
            input= review["review"]
        );

        embedding = embeddingResponse['data'][0]['embedding']

        processed_data.append({
            "values": embedding,
            "id": review["professor"],
            "metadata": {
                "stars": review["stars"],
                "subject": review["subject"],
                "review": review["review"]
            }
        })
    except Exception as e:
        print(f"Error processing review: {review['review']}: {e}")
    
#     # Load a pre-trained LLaMA model
#     # client = Groq(model="llama3-70b-8192", api_key=os.getenv("GROQ_API_KEY"))
#     client = groq.Client(api_key=os.getenv("GROQ_API_KEY"))

Error processing review: Dr. Chen's lectures are engaging and well-structured. She's always willing to help during office hours.: Error code: 405
Error processing review: Prof. Johnson knows his subject well, but his lectures can be dry at times. Assignments are fair.: Error code: 405
Error processing review: Brilliant professor! Dr. Williams explains complex concepts clearly and provides great real-world examples.: Error code: 405
Error processing review: Prof. Lee's teaching style is confusing. Often seems unprepared for class and struggles to answer questions.: Error code: 405
Error processing review: Dr. Rodriguez is passionate about her field and it shows. Her classes are thought-provoking and interactive.: Error code: 405
Error processing review: Prof. Taylor has extensive knowledge but can be intimidating. Grading is strict but fair.: Error code: 405
Error processing review: Dr. Brown is an exceptional teacher. Her labs are well-organized and she explains concepts clearly.: Erro